# Cyber Events Evolution Analysis - Normalized by Internet Usage

## Purpose
This exploratory study examines the **intrinsic evolution of cyber events** by explicitly excluding factors that do not reflect improvements in the capabilities or intentions of cybercriminals.

## Methodology
Rather than simply describing two parallel trends—digitalization and cyber events—this analysis **normalizes the growth in cyber events by an indicator of digital usage** (internet penetration and user count) to better estimate the **intrinsic rise in cyber risk over time**.

### Key Questions
1. **Raw Growth**: How have cyber events increased in absolute terms?
2. **Digitalization Context**: How has internet usage grown in the same period?
3. **Normalized Risk**: What is the intrinsic increase in cyber risk when accounting for increased digital activity?
4. **Comparative Insights**: Do different event types (Advisen, Malware, Ransomware) follow similar normalized trends?

### Datasets Used
- **Internet Usage**: World Bank % of population with internet access (2005-2024)
- **Cyber Events**: Advisen events (2004-2023), Malware attacks (2015-2024), Ransomware attacks (2017-2023)

## 1. Import Required Libraries

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
import scipy.stats as stats
import math

# Style configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print("✓ Libraries successfully imported")
print("  - Data processing: pandas, numpy")
print("  - Visualization: matplotlib, seaborn, plotly")
print("  - Ready for normalized cyber events analysis!")

✓ Libraries successfully imported
  - Data processing: pandas, numpy
  - Visualization: matplotlib, seaborn, plotly
  - Ready for normalized cyber events analysis!


## 1. Load Internet Usage Data (Normalization Factor)

In [146]:
# Construct absolute paths to data folder
current_notebook = Path.cwd()
workspace_root = current_notebook.parent.parent
data_dir = workspace_root / 'data'

# Internet usage data path - World Bank world data
internet_wb_path = data_dir / 'usage' / 'dataworld_bank_world.csv'
world_population_path = data_dir / 'usage' / 'dataworld_bank_world_population.csv'

print(f"📁 Workspace Root: {workspace_root}")
print(f"📁 Data Directory: {data_dir}")
print(f"✓ Internet WB path: {internet_wb_path}")
print(f"✓ World Population path: {world_population_path}")

# World Bank: % of population with internet access (World aggregated data)
# Skip metadata rows (first 4 lines contain metadata, actual data starts at line 4)
df_internet_wb = pd.read_csv(internet_wb_path, skiprows=4)
df_population_wb = pd.read_csv(world_population_path)

# Extract world data - get only World (WLD) data
df_internet_wb_world = df_internet_wb[df_internet_wb['Country Code'] == 'WLD'].copy()
df_population_wb_world = df_population_wb[df_population_wb['Country Code'] == 'WLD'].copy()

# Get all year columns (numeric columns from 1960 onwards)
year_columns = [col for col in df_internet_wb.columns if col.isdigit()]

# Keep only years from 2005 to 2023
year_columns = [col for col in year_columns if int(col) >= 2005 and int(col) < 2024]

# Unpivot to long format
df_internet_wb = df_internet_wb_world[['Country Code'] + year_columns].melt(
    id_vars=['Country Code'],
    var_name='Year',
    value_name='Internet_Penetration_Percent'
)

# Get all year columns (numeric columns from 1960 onwards)
year_columns = [col for col in df_population_wb.columns if col.isdigit()]

# Keep only years from 2005 to 2023
year_columns = [col for col in year_columns if int(col) >= 2005 and int(col) < 2024]
# Unpivot to long format
df_population_wb = df_population_wb_world[['Country Code'] + year_columns].melt(
    id_vars=['Country Code'],
    var_name='Year',
    value_name='World_Population'
)

print("="*70)
print("INTERNET USAGE - WORLD BANK (% of world population)")
print("="*70)
print(f"\n✓ File loaded: {internet_wb_path}")
print(f"  Columns: {df_internet_wb.columns.tolist()}")
print(f"  Data points: {len(df_internet_wb)}")
print(f"\nData:")
#print(df_internet_wb)

print("="*70)
print("WORLD POPULATION - WORLD BANK")
print("="*70)
print(f"\n✓ File loaded: {world_population_path}")
print(f"  Columns: {df_population_wb.columns.tolist()}")
print(f"  Data points: {len(df_population_wb)}")
print(f"\nData:")
print(df_population_wb)

print("="*70)

📁 Workspace Root: c:\Users\jbertholat\Downloads\cyber-hazard-evolution
📁 Data Directory: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data
✓ Internet WB path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\usage\dataworld_bank_world.csv
✓ World Population path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\usage\dataworld_bank_world_population.csv
INTERNET USAGE - WORLD BANK (% of world population)

✓ File loaded: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\usage\dataworld_bank_world.csv
  Columns: ['Country Code', 'Year', 'Internet_Penetration_Percent']
  Data points: 19

Data:
WORLD POPULATION - WORLD BANK

✓ File loaded: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\usage\dataworld_bank_world_population.csv
  Columns: ['Country Code', 'Year', 'World_Population']
  Data points: 19

Data:
   Country Code  Year  World_Population
0           WLD  2005        6575841506
1           WLD  2006        6659977025
2           WLD  2007 

## 1. Load Cyber Events Data

In [147]:
# Load cyber events datasets using absolute paths
advisens_path = data_dir / 'cyber_events' / 'events_Advisen.csv'
statista_path = data_dir / 'cyber_events' / 'statista_cyber_event.csv'
malware_path = data_dir / 'cyber_events' / 'malwares_exploring_topics.csv'
ransomware_path = data_dir / 'cyber_events' / 'randsomware_exploring_topics.csv'

print(f"✓ Advisen path: {advisens_path}")
print(f"✓ Statista path: {statista_path}")
print(f"✓ Malware path: {malware_path}")
print(f"✓ Ransomware path: {ransomware_path}")

# Load Advisen events
df_advisens = pd.read_csv(advisens_path)
# Keep only years from 2004 to 2023
df_advisens = df_advisens[(df_advisens['Date'] > 2004) & (df_advisens['Date'] <= 2023)]

print("\n" + "="*70)
print("CYBER INCIDENTS - ADVISENS (2005-2023)")
print("="*70)
print(f"\n✓ File loaded: {advisens_path}")
print(f"  Columns: {df_advisens.columns.tolist()}")
print(f"  Data points: {len(df_advisens)}")
print(f"  Year range: {df_advisens['Date'].min():.0f} - {df_advisens['Date'].max():.0f}")

# Load Statista events
df_statista = pd.read_csv(statista_path)
# Keep only years from 2005 to 2023
df_statista = df_statista[(df_statista['Date'] >= 2005) & (df_statista['Date'] <= 2023)]
print("\n" + "="*70)
print("CYBER INCIDENTS - STATISTA (2005-2023)")
print("="*70)
print(f"\n✓ File loaded: {statista_path}")
print(f"  Columns: {df_statista.columns.tolist()}")
print(f"  Data points: {len(df_statista)}")
print(f"  Year range: {df_statista['Date'].min():.0f} - {df_statista['Date'].max():.0f}")

# Load Malware data
df_malware = pd.read_csv(malware_path)
print(f"\n" + "="*70)
print("CYBER INCIDENTS - MALWARE (2015-2023)")
df_malware = df_malware[(df_malware['date'] > 2014) & (df_malware['date'] <= 2023)]
print("="*70)
print(f"\n✓ File loaded: {malware_path}")
print(f"  Columns: {df_malware.columns.tolist()}")
print(f"  Year range: {df_malware['date'].min():.0f} - {df_malware['date'].max():.0f}")

# Load Ransomware data
df_ransomware = pd.read_csv(ransomware_path)
print(f"\n" + "="*70)
print("CYBER INCIDENTS - RANSOMWARE (2017-2023)")
print("="*70)
print(f"\n✓ File loaded: {ransomware_path}")
print(f"  Columns: {df_ransomware.columns.tolist()}")
print(f"  Year range: {df_ransomware['Year'].min():.0f} - {df_ransomware['Year'].max():.0f}")

print(f"\n✅ Cyber events data loaded successfully!")

✓ Advisen path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\events_Advisen.csv
✓ Statista path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\statista_cyber_event.csv
✓ Malware path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\malwares_exploring_topics.csv
✓ Ransomware path: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\randsomware_exploring_topics.csv

CYBER INCIDENTS - ADVISENS (2005-2023)

✓ File loaded: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\events_Advisen.csv
  Columns: ['Date', 'nb_cyber_event']
  Data points: 19
  Year range: 2005 - 2023

CYBER INCIDENTS - STATISTA (2005-2023)

✓ File loaded: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\cyber_events\statista_cyber_event.csv
  Columns: ['Date', 'nb_cyber_event_million']
  Data points: 8
  Year range: 2016 - 2023

CYBER INCIDENTS - MALWARE (2015-2023)

✓ File loaded: c:\Users\jberthol

## 2. Data Preparation and Alignment

In [148]:
print("="*70)
print("DATA PREPARATION AND ALIGNMENT")
print("="*70)

# -------- Advisen (en milliers) -> convertir en millions --------
df_advisens_clean = df_advisens.copy()
df_advisens_clean.columns = ['Year', 'Events']
df_advisens_clean['Year'] = df_advisens_clean['Year'].astype(int)
df_advisens_clean['Events'] = pd.to_numeric(df_advisens_clean['Events'], errors='coerce') / 1000000  # 1 -> Million

# -------- Statista (déjà en millions) --------
df_statista_clean = df_statista.copy()
df_statista_clean.columns = ['Year', 'Events']
df_statista_clean['Year'] = df_statista_clean['Year'].astype(int)
df_statista_clean['Events'] = pd.to_numeric(df_statista_clean['Events'], errors='coerce')          # reste en M

# -------- Malware (en milliards) -> convertir en millions --------
df_malware_clean = df_malware.copy()
df_malware_clean.columns = ['Year', 'Malware_Attacks']
df_malware_clean['Year'] = df_malware_clean['Year'].astype(int)
df_malware_clean['Malware_Attacks'] = pd.to_numeric(df_malware_clean['Malware_Attacks'], errors='coerce') * 1000  # B -> M

# -------- Ransomware (déjà en millions) --------
df_ransomware_clean = df_ransomware.copy()
df_ransomware_clean.columns = ['Year', 'Ransomware_Attacks']
df_ransomware_clean['Year'] = df_ransomware_clean['Year'].astype(int)
df_ransomware_clean['Ransomware_Attacks'] = pd.to_numeric(df_ransomware_clean['Ransomware_Attacks'], errors='coerce')  # reste en M

# -------- Internet Usage (World Bank) --------
df_internet_wb['Year'] = df_internet_wb['Year'].astype(int)
df_internet_wb['Internet_Penetration_Percent'] = pd.to_numeric(
    df_internet_wb['Internet_Penetration_Percent'], errors='coerce'
)
df_internet_wb_clean = df_internet_wb[['Year', 'Internet_Penetration_Percent']].dropna()

# -------- World Population (World Bank) --------
df_population_wb['Year'] = df_population_wb['Year'].astype(int)
df_population_wb['World_Population'] = pd.to_numeric(df_population_wb['World_Population'], errors='coerce')
df_population_wb_clean = df_population_wb[['Year', 'World_Population']].dropna()

# -------- Sanity check rapide (toutes les séries sont maintenant en "millions d'événements") --------
for nm, s in [
    ('Advisen (M)', df_advisens_clean['Events']),
    ('Statista (M)', df_statista_clean['Events']),
    ('Malware (M)', df_malware_clean['Malware_Attacks']),
    ('Ransomware (M)', df_ransomware_clean['Ransomware_Attacks']),
]:
    print(f"{nm}: min={s.min():.3f}, median={s.median():.3f}, max={s.max():.3f}")


print(f"\n✓ Advisen: {len(df_advisens_clean)} years ({df_advisens_clean['Year'].min():.0f}-{df_advisens_clean['Year'].max():.0f})")
print(f"✓ Statista: {len(df_statista_clean)} years ({df_statista_clean['Year'].min():.0f}-{df_statista_clean['Year'].max():.0f})")
print(f"✓ Malware: {len(df_malware_clean)} years ({df_malware_clean['Year'].min():.0f}-{df_malware_clean['Year'].max():.0f})")
print(f"✓ Ransomware: {len(df_ransomware_clean)} years ({df_ransomware_clean['Year'].min():.0f}-{df_ransomware_clean['Year'].max():.0f})")
print(f"✓ Internet Penetration (WB - World): {len(df_internet_wb_clean)} years ({df_internet_wb_clean['Year'].min():.0f}-{df_internet_wb_clean['Year'].max():.0f})")
print(f"✓ World Population (WB - World): {len(df_population_wb_clean)} years ({df_population_wb_clean['Year'].min():.0f}-{df_population_wb_clean['Year'].max():.0f})")
print(f"\n✅ Data preparation completed!")

DATA PREPARATION AND ALIGNMENT
Advisen (M): min=0.000, median=0.008, max=0.012
Statista (M): min=4.320, median=13.610, max=19.230
Malware (M): min=5400.000, median=7900.000, max=10500.000
Ransomware (M): min=186.300, median=304.640, max=623.250

✓ Advisen: 19 years (2005-2023)
✓ Statista: 8 years (2016-2023)
✓ Malware: 9 years (2015-2023)
✓ Ransomware: 7 years (2017-2023)
✓ Internet Penetration (WB - World): 19 years (2005-2023)
✓ World Population (WB - World): 19 years (2005-2023)

✅ Data preparation completed!


## 3. Normalization Methodology and Calculations

APPROACH: Normalize cyber events by internet usage growth

1. RAW INCIDENTS: Absolute number/count of events (baseline)
2. INTERNET FACTOR: % of population with internet access (growth context)
3. NORMALIZED INCIDENTS: Raw events adjusted for internet growth

FORMULA:
  Normalized_Events(year) = Events(year) / ((Internet_Penetration(year) / 100)*population)
  
INTERPRETATION:
  - If normalized events decline: cyber security is improving relative to digital exposure
  - If normalized events increase: intrinsic cyber risk is rising
  - If normalized events stay flat: risk growth matches digitalization

In [149]:
print("="*70)
print("NORMALIZATION METHODOLOGY")
print("="*70)

# Merge datasets by year
df_advisens_with_internet = df_advisens_clean.merge(
    df_internet_wb_clean, on='Year', how='inner'
)

df_statista_with_internet = df_statista_clean.merge(
    df_internet_wb_clean, on='Year', how='inner'
)

df_malware_with_internet = df_malware_clean.merge(
    df_internet_wb_clean, on='Year', how='inner'
)

df_ransomware_with_internet = df_ransomware_clean.merge(
    df_internet_wb_clean, on='Year', how='inner'
)

print(f"\n✓ Data merged by common years:")
print(f"  Advisen + Internet: {len(df_advisens_with_internet)} years")
print(f"  Statista + Internet: {len(df_statista_with_internet)} years")
print(f"  Malware + Internet: {len(df_malware_with_internet)} years")
print(f"  Ransomware + Internet: {len(df_ransomware_with_internet)} years")

# Calculate normalized events
df_advisens_with_internet['Events_Normalized'] = (
    df_advisens_with_internet['Events'] / ((df_advisens_with_internet['Internet_Penetration_Percent'] / 100))
)

df_statista_with_internet['Events_Normalized'] = (
    df_statista_with_internet['Events'] / ((df_statista_with_internet['Internet_Penetration_Percent'] / 100))
)

df_malware_with_internet['Malware_Normalized'] = (
    df_malware_with_internet['Malware_Attacks'] / ((df_malware_with_internet['Internet_Penetration_Percent'] / 100))
)

df_ransomware_with_internet['Ransomware_Normalized'] = (
    df_ransomware_with_internet['Ransomware_Attacks'] / ((df_ransomware_with_internet['Internet_Penetration_Percent'] / 100))
)

print(f"\n✓ Normalized events calculated (adjusted for internet penetration)")
print(f"\n📊 ADVISENS ANALYSIS (with normalization):")
print(df_advisens_with_internet[['Year', 'Events', 'Internet_Penetration_Percent', 'Events_Normalized']].to_string())

NORMALIZATION METHODOLOGY

✓ Data merged by common years:
  Advisen + Internet: 19 years
  Statista + Internet: 8 years
  Malware + Internet: 9 years
  Ransomware + Internet: 7 years

✓ Normalized events calculated (adjusted for internet penetration)

📊 ADVISENS ANALYSIS (with normalization):
    Year    Events  Internet_Penetration_Percent  Events_Normalized
0   2005  0.000426                          15.6           0.002730
1   2006  0.001161                          17.2           0.006752
2   2007  0.001510                          20.2           0.007474
3   2008  0.002787                          22.8           0.012224
4   2009  0.002555                          25.3           0.010098
5   2010  0.003097                          28.4           0.010904
6   2011  0.004258                          30.9           0.013780
7   2012  0.005729                          33.3           0.017204
8   2013  0.009213                          35.3           0.026099
9   2014  0.008903        

## 3v. Visualization - Raw vs Normalized Events

In [150]:
# Create overlay chart: Raw vs Normalized events on same plot with dual axes
fig_analysis = go.Figure()
scale_factor = 1000000.0

# Add raw events (left axis)
fig_analysis.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events']*scale_factor,
    mode='lines+markers',
    name='Raw Events',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8),
    yaxis='y1',
    hovertemplate='<b>Year: %{x}</b><br>Raw Events: %{y:,.0f}<extra></extra>'
))

# Add normalized Advisen's events (right axis)
fig_analysis.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events_Normalized']*scale_factor,
    mode='lines+markers',
    name='Advisen Normalized Events',
    line=dict(color='#ff7f0e', width=3, dash='dash'),
    marker=dict(size=8, symbol='diamond'),
    yaxis='y2',
    hovertemplate='<b>Year: %{x}</b><br>Normalized: %{y:,.0f}<extra></extra>'
))

# Update layout with dual axes
fig_analysis.update_layout(
    title={
        'text': '<b>Cyber Events: Raw vs Advisen Normalized by Internet Penetration (2005-2023)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=2,
        tickangle=-45
    ),
    yaxis=dict(
        title='<b>Raw Events</b>',
        title_font=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4')
    ),
    yaxis2=dict(
        title='<b>Advisen Normalized Events</b>',
        title_font=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right'
    ),
    width=1200,
    height=600,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=150, b=100)
)

# Save outputs
output_dir = Path('./outputs')
output_dir.mkdir(parents=True, exist_ok=True)

fig_analysis.write_html(output_dir / '03_advisens_raw_vs_normalized.html')
fig_analysis.write_image(output_dir / '03_advisens_raw_vs_normalized.png', width=1200, height=600)

print("✓ Chart saved: ./outputs/03_advisens_raw_vs_normalized.html")
print("✓ Chart saved: ./outputs/03_advisens_raw_vs_normalized.png")

fig_analysis.show()



✓ Chart saved: ./outputs/03_advisens_raw_vs_normalized.html
✓ Chart saved: ./outputs/03_advisens_raw_vs_normalized.png


In [151]:
# Create overlay chart: Raw vs Normalized events on same plot with dual axes
fig_analysis = go.Figure()

# Add raw events (left axis)
fig_analysis.add_trace(go.Scatter(
    x=df_statista_with_internet['Year'],
    y=df_statista_with_internet['Events'],
    mode='lines+markers',
    name='Raw Events',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8),
    yaxis='y1',
    hovertemplate='<b>Year: %{x}</b><br>Raw Events: %{y:,.0f}<extra></extra>'
))

# Add normalized statista dataset events (right axis)
fig_analysis.add_trace(go.Scatter(
    x=df_statista_with_internet['Year'],
    y=df_statista_with_internet['Events_Normalized'],
    mode='lines+markers',
    name='Statista Normalized Events',
    line=dict(color='#2ca02c', width=3, dash='dot'),
    marker=dict(size=8, symbol='square'),
    yaxis='y2',
    hovertemplate='<b>Year: %{x}</b><br>Normalized: %{y:,.0f}<extra></extra>'
))

# Update layout with dual axes
fig_analysis.update_layout(
    title={
        'text': '<b>Cyber Events: Raw vs Normalized by Internet Penetration (2005-2023)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=2,
        tickangle=-45
    ),
    yaxis=dict(
        title='<b>Raw Events (Millions)</b>',
        title_font=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4')
    ),
    yaxis2=dict(
        title='<b>Statista Normalized Events</b><br>(Millions)',
        title_font=dict(color='#2ca02c'),
        tickfont=dict(color='#2ca02c'),
        overlaying='y',
        side='right'
    ),
    width=1200,
    height=600,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=150, b=100)
)

# Save outputs
output_dir = Path('./outputs')
output_dir.mkdir(parents=True, exist_ok=True)

fig_analysis.write_html(output_dir / '03_statista_raw_vs_normalized.html')
fig_analysis.write_image(output_dir / '03_statista_raw_vs_normalized.png', width=1200, height=600)

print("✓ Chart saved: ./outputs/03_statista_raw_vs_normalized.html")
print("✓ Chart saved: ./outputs/03_statista_raw_vs_normalized.png")

fig_analysis.show()


✓ Chart saved: ./outputs/03_statista_raw_vs_normalized.html
✓ Chart saved: ./outputs/03_statista_raw_vs_normalized.png


In [152]:
# Create overlay figure: Advisen and Statista Normalized Events comparison with dual axes
print("\n" + "="*70)
print("NORMALIZED INCIDENTS OVERLAY: Advisen vs Statista")
print("="*70)

fig_analysis = go.Figure()

# Add Advisen Normalized events (orange) - left axis
fig_analysis.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events_Normalized'],
    mode='lines+markers',
    name='Advisen (Normalized)',
    line=dict(color='#ff7f0e', width=3),
    marker=dict(size=8),
    yaxis='y1',
    hovertemplate='<b>Year: %{x}</b><br>Advisen Normalized: %{y:,.2f}<extra></extra>'
))

# Add Statista Normalized events (green) - right axis
fig_analysis.add_trace(go.Scatter(
    x=df_statista_with_internet['Year'],
    y=df_statista_with_internet['Events_Normalized'],
    mode='lines+markers',
    name='Statista (Normalized)',
    line=dict(color='#2ca02c', width=3),
    marker=dict(size=8),
    yaxis='y2',
    hovertemplate='<b>Year: %{x}</b><br>Statista Normalized: %{y:,.2f}<extra></extra>'
))

fig_analysis.update_layout(
    title={
        'text': '<b>Normalized Cyber Events Overlay: Advisen vs Statista</b><br><sub>Adjusted by Internet Penetration (Dual Axes for Visual Alignment)</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=1,
        tickangle=-45
    ),
    yaxis=dict(
        title='<b>Advisen Normalized (Millions)</b>',
        title_font=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#e8e8e8'
    ),
    yaxis2=dict(
        title='<b>Statista Normalized</b><br>(Millions)',
        title_font=dict(color='#2ca02c'),
        tickfont=dict(color='#2ca02c'),
        overlaying='y',
        side='right'
    ),
    width=1200,
    height=650,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=180, b=100),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_analysis.write_html(output_dir / '03_normalized_advisens_statista_overlay.html')
fig_analysis.write_image(output_dir / '03_normalized_advisens_statista_overlay.png', width=1200, height=650)

print("\n✓ Chart saved: ./outputs/03_normalized_advisens_statista_overlay.html")
print("✓ Chart saved: ./outputs/03_normalized_advisens_statista_overlay.png")

fig_analysis.show()


NORMALIZED INCIDENTS OVERLAY: Advisen vs Statista

✓ Chart saved: ./outputs/03_normalized_advisens_statista_overlay.html
✓ Chart saved: ./outputs/03_normalized_advisens_statista_overlay.png


# 4. Visualisation Normalized  cyber event evolution (%) per year 

- **Raw % Evolution** : Raw number of events % evolution (no normalization)
- **% Evolution (Normalized)** : % of evolution of normalized events

In [153]:
print("="*70)
print("7. NORMALIZED CYBER INCIDENT EVOLUTION (%) PER YEAR - MULTI-DATASET")
print("="*70)

# Calculate year-over-year percentage evolution for all three datasets
df_advisens_sorted = df_advisens_with_internet.sort_values('Year').reset_index(drop=True)
df_statista_sorted = df_statista_with_internet.sort_values('Year').reset_index(drop=True)
df_malware_sorted = df_malware_with_internet.sort_values('Year').reset_index(drop=True)
df_ransomware_sorted = df_ransomware_with_internet.sort_values('Year').reset_index(drop=True)

# Calculate Advisen evolution
advisens_years = []
advisens_pct_evolution = []

for i in range(1, len(df_advisens_sorted)):
    current_year = df_advisens_sorted.loc[i, 'Year']
    current_norm = df_advisens_sorted.loc[i, 'Events_Normalized']
    prev_norm = df_advisens_sorted.loc[i-1, 'Events_Normalized']
    
    if prev_norm != 0:
        pct_evolution = ((current_norm - prev_norm) / prev_norm) * 100
    else:
        pct_evolution = 0
    
    advisens_years.append(current_year)
    advisens_pct_evolution.append(pct_evolution)
    
    
# Calculate Statista evolution
statista_years = []
statista_pct_evolution = []

for i in range(1, len(df_statista_sorted)):
    current_year = df_statista_sorted.loc[i, 'Year']
    current_norm = df_statista_sorted.loc[i, 'Events_Normalized']
    prev_norm = df_statista_sorted.loc[i-1, 'Events_Normalized']

    if prev_norm != 0:
        pct_evolution = ((current_norm - prev_norm) / prev_norm) * 100
    else:
        pct_evolution = 0

    statista_years.append(current_year)
    statista_pct_evolution.append(pct_evolution)

# Calculate Malware evolution
malware_years = []
malware_pct_evolution = []

for i in range(1, len(df_malware_sorted)):
    current_year = df_malware_sorted.loc[i, 'Year']
    current_norm = df_malware_sorted.loc[i, 'Malware_Normalized']
    prev_norm = df_malware_sorted.loc[i-1, 'Malware_Normalized']
    
    if prev_norm != 0:
        pct_evolution = ((current_norm - prev_norm) / prev_norm) * 100
    else:
        pct_evolution = 0
    
    malware_years.append(current_year)
    malware_pct_evolution.append(pct_evolution)

# Calculate Ransomware evolution
ransomware_years = []
ransomware_pct_evolution = []

for i in range(1, len(df_ransomware_sorted)):
    current_year = df_ransomware_sorted.loc[i, 'Year']
    current_norm = df_ransomware_sorted.loc[i, 'Ransomware_Normalized']
    prev_norm = df_ransomware_sorted.loc[i-1, 'Ransomware_Normalized']
    
    if prev_norm != 0:
        pct_evolution = ((current_norm - prev_norm) / prev_norm) * 100
    else:
        pct_evolution = 0
    
    ransomware_years.append(current_year)
    ransomware_pct_evolution.append(pct_evolution)

print(f"\n✓ Calculations completed:")
print(f"  Advisen: {len(advisens_years)} year-over-year transitions (2005-2023)")
print(f"  Statista: {len(statista_years)} year-over-year transitions (2005-2023)")
print(f"  Malware: {len(malware_years)} year-over-year transitions (2016-2024)")
print(f"  Ransomware: {len(ransomware_years)} year-over-year transitions (2018-2023)")

# Create grouped bar chart for normalized events evolution
fig_evolution = go.Figure()

# Add Advisen bars (blue)
fig_evolution.add_trace(go.Bar(
    x=advisens_years,
    y=advisens_pct_evolution,
    name='Advisen (Normalized)',
    marker=dict(color='#1f77b4'),
    hovertemplate='<b>Advisen - Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

fig_evolution.add_trace(go.Bar(
    x=statista_years,
    y=statista_pct_evolution,
    name='Statista (Normalized)',
    marker=dict(color='#2ca02c'),
    hovertemplate='<b>Statista - Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

# Add Malware bars (orange)
fig_evolution.add_trace(go.Bar(
    x=malware_years,
    y=malware_pct_evolution,
    name='Malware (Normalized)',
    marker=dict(color='#ff7f0e'),
    hovertemplate='<b>Malware - Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

# Add Ransomware bars (red)
fig_evolution.add_trace(go.Bar(
    x=ransomware_years,
    y=ransomware_pct_evolution,
    name='Ransomware (Normalized)',
    marker=dict(color='#d62728'),
    hovertemplate='<b>Ransomware - Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

# Add a reference line at 0%
fig_evolution.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1, annotation_text="No evolution (0%)")

fig_evolution.update_layout(
    title={
        'text': '<b>Normalized Cyber Events: Year-over-Year Evolution (%) - All Datasets</b><br><sub>Percentage evolution in normalized events from previous year</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=1,
        tickangle=-45
    ),
    yaxis=dict(
        title='Year-over-Year % evolution',
        showgrid=True,
        gridwidth=1,
        gridcolor='#e8e8e8'
    ),
    barmode='group',
    width=1400,
    height=700,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=180, b=100),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_evolution.write_html(output_dir / '04_normalized_evolution_percent_all_datasets.html')
fig_evolution.write_image(output_dir / '04_normalized_evolution_percent_all_datasets.png', width=1400, height=700)

print("\n✓ Chart saved: ./outputs/04_normalized_evolution_percent_all_datasets.html")
print("✓ Chart saved: ./outputs/04_normalized_evolution_percent_all_datasets.png")

# Print summary statistics
print(f"\n" + "="*70)
print("EVOLUTION ANALYSIS SUMMARY - ALL DATASETS")
print("="*70)

print(f"\n📊 ADVISENS NORMALIZED INCIDENTS (2005-2023):")
print(f"  Average annual % evolution: {np.mean(advisens_pct_evolution):+.1f}%")
print(f"  Max increase year: {advisens_years[np.argmax(advisens_pct_evolution)]:.0f} ({max(advisens_pct_evolution):+.1f}%)")
print(f"  Max decrease year: {advisens_years[np.argmin(advisens_pct_evolution)]:.0f} ({min(advisens_pct_evolution):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in advisens_pct_evolution if x > 0)}/{len(advisens_pct_evolution)}")
print(f"  Years with negative growth: {sum(1 for x in advisens_pct_evolution if x < 0)}/{len(advisens_pct_evolution)}")

print(f"\n📊 STATISTA NORMALIZED INCIDENTS (2005-2023):")
print(f"  Average annual % evolution: {np.mean(statista_pct_evolution):+.1f}%")
print(f"  Max increase year: {statista_years[np.argmax(statista_pct_evolution)]:.0f} ({max(statista_pct_evolution):+.1f}%)")
print(f"  Max decrease year: {statista_years[np.argmin(statista_pct_evolution)]:.0f} ({min(statista_pct_evolution):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in statista_pct_evolution if x > 0)}/{len(statista_pct_evolution)}")
print(f"  Years with negative growth: {sum(1 for x in statista_pct_evolution if x < 0)}/{len(statista_pct_evolution)}")

print(f"\n📊 MALWARE NORMALIZED ATTACKS (2016-2024):")
print(f"  Average annual % evolution: {np.mean(malware_pct_evolution):+.1f}%")
print(f"  Max increase year: {malware_years[np.argmax(malware_pct_evolution)]:.0f} ({max(malware_pct_evolution):+.1f}%)")
print(f"  Max decrease year: {malware_years[np.argmin(malware_pct_evolution)]:.0f} ({min(malware_pct_evolution):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in malware_pct_evolution if x > 0)}/{len(malware_pct_evolution)}")
print(f"  Years with negative growth: {sum(1 for x in malware_pct_evolution if x < 0)}/{len(malware_pct_evolution)}")

print(f"\n📊 RANSOMWARE NORMALIZED ATTACKS (2018-2023):")
print(f"  Average annual % evolution: {np.mean(ransomware_pct_evolution):+.1f}%")
print(f"  Max increase year: {ransomware_years[np.argmax(ransomware_pct_evolution)]:.0f} ({max(ransomware_pct_evolution):+.1f}%)")
print(f"  Max decrease year: {ransomware_years[np.argmin(ransomware_pct_evolution)]:.0f} ({min(ransomware_pct_evolution):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in ransomware_pct_evolution if x > 0)}/{len(ransomware_pct_evolution)}")
print(f"  Years with negative growth: {sum(1 for x in ransomware_pct_evolution if x < 0)}/{len(ransomware_pct_evolution)}")

fig_evolution.show()

7. NORMALIZED CYBER INCIDENT EVOLUTION (%) PER YEAR - MULTI-DATASET

✓ Calculations completed:
  Advisen: 18 year-over-year transitions (2005-2023)
  Statista: 7 year-over-year transitions (2005-2023)
  Malware: 8 year-over-year transitions (2016-2024)
  Ransomware: 6 year-over-year transitions (2018-2023)

✓ Chart saved: ./outputs/04_normalized_evolution_percent_all_datasets.html
✓ Chart saved: ./outputs/04_normalized_evolution_percent_all_datasets.png

EVOLUTION ANALYSIS SUMMARY - ALL DATASETS

📊 ADVISENS NORMALIZED INCIDENTS (2005-2023):
  Average annual % evolution: +15.9%
  Max increase year: 2006 (+147.4%)
  Max decrease year: 2017 (-20.6%)
  Years with positive growth: 11/18
  Years with negative growth: 7/18

📊 STATISTA NORMALIZED INCIDENTS (2005-2023):
  Average annual % evolution: +16.4%
  Max increase year: 2020 (+55.5%)
  Max decrease year: 2022 (-15.4%)
  Years with positive growth: 5/7
  Years with negative growth: 2/7

📊 MALWARE NORMALIZED ATTACKS (2016-2024):
  Average 

In [154]:
print("="*70)
print("7. RAW CYBER INCIDENT EVOLUTION (%) PER YEAR - ADVISENS")
print("="*70)

# Calculate year-over-year percentage evolution for Advisen raw events
df_advisens_sorted = df_advisens_with_internet.sort_values('Year').reset_index(drop=True)

# Initialize columns for calculations
years_for_evolution = []
pct_evolution_raw = []

print(f"\n📊 Year-over-Year Evolution Calculations:")
print(f"{'Year':<8} {'Year-1 Raw':<15} {'Year Raw':<15} {'% evolution':<12} {'Raw % evolution':<15}")
print("-" * 75)

# Calculate percentage evolution for each year (starting from the second year)
for i in range(1, len(df_advisens_sorted)):
    current_year = df_advisens_sorted.loc[i, 'Year']
    prev_year = df_advisens_sorted.loc[i-1, 'Year']

    current_raw = df_advisens_sorted.loc[i, 'Events']
    prev_raw = df_advisens_sorted.loc[i-1, 'Events']
    
    # Calculate percentage evolution (handling potential division by zero)
    if prev_raw != 0:
        pct_evolution_r = ((current_raw - prev_raw) / prev_raw) * 100
    else:
        pct_evolution_r = 0
    
    years_for_evolution.append(current_year)
    pct_evolution_raw.append(pct_evolution_r)

    #print(f"{current_year:<8} {prev_raw:<15.2f} {current_raw:<15.2f} {pct_evolution_raw:>+10.1f}% {pct_evolution_r:>+13.1f}%")

print(f"\n✓ Calculations completed for {len(years_for_evolution)} year-over-year transitions")

# Create bar chart for raw events evolution
fig_evolution = go.Figure()

# Add raw percentage evolution bars
fig_evolution.add_trace(go.Bar(
    x=years_for_evolution,
    y=pct_evolution_raw,
    name='Raw Events % evolution',
    marker=dict(
        color=pct_evolution_raw,
        colorscale='RdYlGn_r',  # Red for negative (improvement), Green for positive (deterioration)
        showscale=True,
        colorbar=dict(title="% evolution")
    ),
    hovertemplate='<b>Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

# Add a reference line at 0%
fig_evolution.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="No evolution (0%)")

fig_evolution.update_layout(
    title={
        'text': '<b>Advisen Raw Events: Year-over-Year Evolution (%)</b><br><sub>Percentage evolution in raw events from previous year</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=1,
        tickangle=-45
    ),
    yaxis=dict(
        title='Year-over-Year % evolution',
        showgrid=True,
        gridwidth=1,
        gridcolor='#e8e8e8'
    ),
    width=1200,
    height=650,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=180, b=100),
    showlegend=False
)

fig_evolution.write_html(output_dir / '04_normalized_evolution_percent_advisens.html')
fig_evolution.write_image(output_dir / '04_normalized_evolution_percent_advisens.png', width=1200, height=650)

print("\n✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.html")
print("✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.png")

fig_evolution.show()

print(f"\n" + "="*70)
print("EVOLUTION ANALYSIS SUMMARY")
print("="*70)
print(f"\nAdvisen Raw Events Evolution (2005-2023):")
print(f"  Average annual % evolution: {np.mean(pct_evolution_raw):+.1f}%")
print(f"  Max increase year: {years_for_evolution[np.argmax(pct_evolution_raw)]:.0f} ({max(pct_evolution_raw):+.1f}%)")
print(f"  Max decrease year: {years_for_evolution[np.argmin(pct_evolution_raw)]:.0f} ({min(pct_evolution_raw):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in pct_evolution_raw if x > 0)}/{len(pct_evolution_raw)}")
print(f"  Years with negative growth: {sum(1 for x in pct_evolution_raw if x < 0)}/{len(pct_evolution_raw)}")

7. RAW CYBER INCIDENT EVOLUTION (%) PER YEAR - ADVISENS

📊 Year-over-Year Evolution Calculations:
Year     Year-1 Raw      Year Raw        % evolution  Raw % evolution
---------------------------------------------------------------------------

✓ Calculations completed for 18 year-over-year transitions

✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.html
✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.png



EVOLUTION ANALYSIS SUMMARY

Advisen Raw Events Evolution (2005-2023):
  Average annual % evolution: +26.0%
  Max increase year: 2006 (+172.7%)
  Max decrease year: 2017 (-16.2%)
  Years with positive growth: 11/18
  Years with negative growth: 7/18


In [155]:
print("="*70)
print("7. NORMALIZED CYBER INCIDENT EVOLUTION (%) PER YEAR - ADVISENS")
print("="*70)

# Calculate year-over-year percentage evolution for Advisen normalized events
df_advisens_sorted = df_advisens_with_internet.sort_values('Year').reset_index(drop=True)

# Initialize columns for calculations
years_for_evolution = []
pct_evolution_normalized = []
pct_evolution_raw = []

print(f"\n📊 Year-over-Year Evolution Calculations:")
print(f"{'Year':<8} {'Year-1 Norm':<15} {'Year Norm':<15} {'% evolution':<12} {'Raw % evolution':<15}")
print("-" * 75)

# Calculate percentage evolution for each year (starting from the second year)
for i in range(1, len(df_advisens_sorted)):
    current_year = df_advisens_sorted.loc[i, 'Year']
    prev_year = df_advisens_sorted.loc[i-1, 'Year']
    
    current_norm = df_advisens_sorted.loc[i, 'Events_Normalized']
    prev_norm = df_advisens_sorted.loc[i-1, 'Events_Normalized']
    
    current_raw = df_advisens_sorted.loc[i, 'Events']
    prev_raw = df_advisens_sorted.loc[i-1, 'Events']
    
    # Calculate percentage evolution (handling potential division by zero)
    if prev_norm != 0:
        pct_evolution_norm = ((current_norm - prev_norm) / prev_norm) * 100
    else:
        pct_evolution_norm = 0
    
    if prev_raw != 0:
        pct_evolution_r = ((current_raw - prev_raw) / prev_raw) * 100
    else:
        pct_evolution_r = 0
    
    years_for_evolution.append(current_year)
    pct_evolution_normalized.append(pct_evolution_norm)
    pct_evolution_raw.append(pct_evolution_r)
    
    print(f"{current_year:<8} {prev_norm:<15.2f} {current_norm:<15.2f} {pct_evolution_norm:>+10.1f}% {pct_evolution_r:>+13.1f}%")

print(f"\n✓ Calculations completed for {len(years_for_evolution)} year-over-year transitions")

# Create bar chart for normalized events evolution
fig_evolution = go.Figure()

# Add normalized percentage evolution bars
fig_evolution.add_trace(go.Bar(
    x=years_for_evolution,
    y=pct_evolution_normalized,
    name='Normalized Events % evolution',
    marker=dict(
        color=pct_evolution_normalized,
        colorscale='RdYlGn_r',  # Red for negative (improvement), Green for positive (deterioration)
        showscale=True,
        colorbar=dict(title="% evolution")
    ),
    hovertemplate='<b>Year: %{x}</b><br>% evolution: %{y:+.1f}%<extra></extra>'
))

# Add a reference line at 0%
fig_evolution.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="No evolution (0%)")

fig_evolution.update_layout(
    title={
        'text': '<b>Advisen Normalized Events: Year-over-Year Evolution (%)</b><br><sub>Percentage evolution in normalized events from previous year</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=1,
        tickangle=-45
    ),
    yaxis=dict(
        title='Year-over-Year % evolution',
        showgrid=True,
        gridwidth=1,
        gridcolor='#e8e8e8'
    ),
    width=1200,
    height=650,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=180, b=100),
    showlegend=False
)

fig_evolution.write_html(output_dir / '04_normalized_evolution_percent_advisens.html')
fig_evolution.write_image(output_dir / '04_normalized_evolution_percent_advisens.png', width=1200, height=650)

print("\n✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.html")
print("✓ Chart saved: ./outputs/04_normalized_evolution_percent_advisens.png")

fig_evolution.show()

print(f"\n" + "="*70)
print("EVOLUTION ANALYSIS SUMMARY")
print("="*70)
print(f"\nAdvisen Normalized Events Evolution (2005-2023):")
print(f"  Average annual % evolution: {np.mean(pct_evolution_normalized):+.1f}%")
print(f"  Max increase year: {years_for_evolution[np.argmax(pct_evolution_normalized)]:.0f} ({max(pct_evolution_normalized):+.1f}%)")
print(f"  Max decrease year: {years_for_evolution[np.argmin(pct_evolution_normalized)]:.0f} ({min(pct_evolution_normalized):+.1f}%)")
print(f"  Years with positive growth: {sum(1 for x in pct_evolution_normalized if x > 0)}/{len(pct_evolution_normalized)}")
print(f"  Years with negative growth: {sum(1 for x in pct_evolution_normalized if x < 0)}/{len(pct_evolution_normalized)}")

7. NORMALIZED CYBER INCIDENT EVOLUTION (%) PER YEAR - ADVISENS

📊 Year-over-Year Evolution Calculations:
Year     Year-1 Norm     Year Norm       % evolution  Raw % evolution
---------------------------------------------------------------------------
2006     0.00            0.01                +147.4%        +172.7%
2007     0.01            0.01                 +10.7%         +30.0%
2008     0.01            0.01                 +63.6%         +84.6%
2009     0.01            0.01                 -17.4%          -8.3%
2010     0.01            0.01                  +8.0%         +21.2%
2011     0.01            0.01                 +26.4%         +37.5%
2012     0.01            0.02                 +24.8%         +34.5%
2013     0.02            0.03                 +51.7%         +60.8%
2014     0.03            0.02                  -8.8%          -3.4%
2015     0.02            0.02                 -12.6%          -7.0%
2016     0.02            0.02                 +13.0%         +21.5%
2


EVOLUTION ANALYSIS SUMMARY

Advisen Normalized Events Evolution (2005-2023):
  Average annual % evolution: +15.9%
  Max increase year: 2006 (+147.4%)
  Max decrease year: 2017 (-20.6%)
  Years with positive growth: 11/18
  Years with negative growth: 7/18


Interprétation des résultats

Le nombre d’events cyber normalisés (corrigés de la croissance de l’usage d’Internet) montre une forte hausse initiale au milieu des années 2000 (notamment en 2006), signe d’une intensification rapide du risque à cette époque.

Après cette phase de croissance très marquée, la tendance devient plus modérée et irrégulière : certaines années affichent encore des hausses (ex. 2008, 2013, 2020), mais d’autres montrent des baisses nettes.

Globalement, depuis la seconde moitié des années 2010, les variations sont plus faibles et souvent négatives, indiquant une stabilisation ou un ralentissement du risque relatif.

Cela suggère que, bien que le nombre brut d’events continue d’augmenter, cette hausse est largement expliquée par la croissance du nombre d’utilisateurs plutôt que par une dégradation intrinsèque du niveau de sécurité.

En d’autres termes : le risque individuel moyen (par utilisateur ou par unité d’exposition) n’augmente plus fortement ; il pourrait même s’améliorer légèrement selon certaines années.

## 5a. Visualization - Overlay: Events vs Internet Growth (No usefull)

In [156]:
# Create overlay chart: events vs internet penetration
fig_overlay = go.Figure()
scale_factor = 1000000.0

# Add raw events
fig_overlay.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events'] * scale_factor,
    mode='lines+markers',
    name='Advisen Events',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8),
    yaxis='y1',
    hovertemplate='<b>Year: %{x}</b><br>Events: %{y:,.0f}<extra></extra>'
))

# Add internet penetration on secondary axis
fig_overlay.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Internet_Penetration_Percent'],
    mode='lines+markers',
    name='Internet Penetration %',
    line=dict(color='#2ca02c', width=3, dash='dash'),
    marker=dict(size=8, symbol='diamond'),
    yaxis='y2',
    hovertemplate='<b>Year: %{x}</b><br>Penetration: %{y:.1f}%<extra></extra>'
))

fig_overlay.update_layout(
    title={
        'text': '<b>Advisen Events vs Internet Penetration Growth (2005-2023)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=2,
        tickangle=-45
    ),
    yaxis=dict(
        title='<b>Advisen Events</b>',
        title_font=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4')
    ),
    yaxis2=dict(
        title='<b>Internet Penetration (%)</b>',
        title_font=dict(color='#2ca02c'),
        tickfont=dict(color='#2ca02c'),
        overlaying='y',
        side='right'
    ),
    width=1200,
    height=600,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=150, b=100)
)

fig_overlay.write_html(output_dir / '05a_advisens_with_internet_overlay.html')
fig_overlay.write_image(output_dir / '05a_advisens_with_internet_overlay.png', width=1200, height=600)

print("✓ Chart saved: ./outputs/05a_advisens_with_internet_overlay.html")
print("✓ Chart saved: ./outputs/05a_advisens_with_internet_overlay.png")

fig_overlay.show()

✓ Chart saved: ./outputs/05a_advisens_with_internet_overlay.html
✓ Chart saved: ./outputs/05a_advisens_with_internet_overlay.png


## 5b. Multi-Dataset Comparison - Normalized Events (No usefull)

In [157]:
# Create comparison chart for all three event types (normalized)
fig_multi = go.Figure()

# Add normalized Advisen (2004-2023)
fig_multi.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events_Normalized'],
    mode='lines+markers',
    name='Advisen (Normalized)',
    line=dict(color='#1f77b4', width=2.5),
    marker=dict(size=6),
    hovertemplate='<b>Year: %{x}</b><br>Events: %{y:,.0f}<extra></extra>'
))

fig_multi.add_trace(go.Scatter(
    x=df_statista_with_internet['Year'],
    y=df_statista_with_internet['Events_Normalized'],
    mode='lines+markers',
    name='Statista (Normalized)',
    line=dict(color='#2ca02c', width=2.5),
    marker=dict(size=6),
    hovertemplate='<b>Year: %{x}</b><br>Events: %{y:,.0f}<extra></extra>'
))

# Add normalized Malware (2015-2024)
fig_multi.add_trace(go.Scatter(
    x=df_malware_with_internet['Year'],
    y=df_malware_with_internet['Malware_Normalized'],
    mode='lines+markers',
    name='Malware (Normalized)',
    line=dict(color='#ff7f0e', width=2.5),
    marker=dict(size=6),
    hovertemplate='<b>Year: %{x}</b><br>Malware: %{y:.2f}B<extra></extra>'
))

# Add normalized Ransomware (2017-2023)
fig_multi.add_trace(go.Scatter(
    x=df_ransomware_with_internet['Year'],
    y=df_ransomware_with_internet['Ransomware_Normalized'],
    mode='lines+markers',
    name='Ransomware (Normalized)',
    line=dict(color='#d62728', width=2.5),
    marker=dict(size=6),
    hovertemplate='<b>Year: %{x}</b><br>Ransomware: %{y:,.0f}<extra></extra>'
))

fig_multi.update_layout(
    title={
        'text': '<b>Normalized Cyber Events Comparison (by Internet Penetration)</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#222222'}
    },
    xaxis=dict(
        title='Year',
        dtick=1,
        tickangle=-45
    ),
    yaxis=dict(
        title='Normalized Event Count (Millions)',
        type='log',  # Log scale to compare different magnitudes
        showgrid=True,
        gridwidth=1,
        gridcolor='#e8e8e8'
    ),
    width=1200,
    height=700,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=150, b=150),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_multi.write_html(output_dir / '05b_all_events_normalized_comparison.html')
fig_multi.write_image(output_dir / '05b_all_events_normalized_comparison.png', width=1200, height=700)

print("✓ Chart saved: ./outputs/05b_all_events_normalized_comparison.html")
print("✓ Chart saved: ./outputs/05b_all_events_normalized_comparison.png")

fig_multi.show()

✓ Chart saved: ./outputs/05b_all_events_normalized_comparison.html
✓ Chart saved: ./outputs/05b_all_events_normalized_comparison.png


## 6. Advanced Comparative Analysis Methods (6a, 6b, 6c)

### Overview: Three Complementary Approaches

To deeply understand the intrinsic evolution of cyber risk beyond internet growth, we implement three comparative methodologies:

1. **6a - Simple Normalization**: Direct ratio analysis of events per internet user
2. **6b - Multivariate Regression (Elasticity)**: Log-log regression to measure event elasticity
3. **6c - Residual Analysis**: Study structural risk evolution at constant exposure

Each approach reveals different aspects of the cyber threat landscape.

### 6a - Simple Normalization: Events per Internet User (same as first analsis but different scale)

**Methodology**: Normalize events by dividing by the estimated number of internet users (internet penetration × world population).

**Formula**:
- Events_per_user = Events / (Internet_Penetration / 100 × World_Population)

**Interpretation**:
- Decreasing ratio → Cyber security improving (fewer events per connected user)
- Constant ratio → Risk proportional to exposure
- Increasing ratio → Structural deterioration in cyber risk

In [158]:
print("="*70)
print("6A. SIMPLE NORMALIZATION: Events per Internet User")
print("="*70)

# Load world population data from World Bank CSV
world_population_path = data_dir / 'usage' / 'dataworld_bank_world_population.csv'
df_population = pd.read_csv(world_population_path)

# Extract World population data (country code 'WLD')
df_world_pop = df_population[df_population['Country Code'] == 'WLD'].copy()

# Get year columns
year_columns = [col for col in df_world_pop.columns if col.isdigit()]

# Create a dictionary mapping year to population
world_population = {}
for year in year_columns:
    year_int = int(year)
    if 2004 <= year_int <= 2024:
        pop_value = pd.to_numeric(df_world_pop[year].values[0], errors='coerce')
        if not pd.isna(pop_value):
            world_population[year_int] = pop_value

print(f"✓ World population data loaded from: {world_population_path}")
print(f"  Years available: {min(world_population.keys())} - {max(world_population.keys())}")

# Calculate internet users for each year
df_advisens_with_internet['Internet_Users'] = (
    df_advisens_with_internet['Year'].map(world_population) * 
    (df_advisens_with_internet['Internet_Penetration_Percent'] / 100)
)

df_malware_with_internet['Internet_Users'] = (
    df_malware_with_internet['Year'].map(world_population) * 
    (df_malware_with_internet['Internet_Penetration_Percent'] / 100)
)

df_ransomware_with_internet['Internet_Users'] = (
    df_ransomware_with_internet['Year'].map(world_population) * 
    (df_ransomware_with_internet['Internet_Penetration_Percent'] / 100)
)

# Calculate events per user
df_advisens_with_internet['Events_per_User'] = (
    df_advisens_with_internet['Events'] / df_advisens_with_internet['Internet_Users']
)

df_malware_with_internet['Malware_per_User'] = (
    df_malware_with_internet['Malware_Attacks'] / df_malware_with_internet['Internet_Users']
)

df_ransomware_with_internet['Ransomware_per_User'] = (
    df_ransomware_with_internet['Ransomware_Attacks'] / df_ransomware_with_internet['Internet_Users']
)

# Calculate trends
advisens_per_user_start = df_advisens_with_internet.iloc[0]['Events_per_User']
advisens_per_user_end = df_advisens_with_internet.iloc[-1]['Events_per_User']
advisens_per_user_growth = ((advisens_per_user_end - advisens_per_user_start) / advisens_per_user_start * 100)

malware_per_user_start = df_malware_with_internet.iloc[0]['Malware_per_User']
malware_per_user_end = df_malware_with_internet.iloc[-1]['Malware_per_User']
malware_per_user_growth = ((malware_per_user_end - malware_per_user_start) / malware_per_user_start * 100)

ransomware_per_user_start = df_ransomware_with_internet.iloc[0]['Ransomware_per_User']
ransomware_per_user_end = df_ransomware_with_internet.iloc[-1]['Ransomware_per_User']
ransomware_per_user_growth = ((ransomware_per_user_end - ransomware_per_user_start) / ransomware_per_user_start * 100)

# Add Statista per-user calculation
df_statista_with_internet['Internet_Users'] = (
    df_statista_with_internet['Year'].map(world_population) * 
    (df_statista_with_internet['Internet_Penetration_Percent'] / 100)
)

df_statista_with_internet['Events_per_User'] = (
    df_statista_with_internet['Events'] / df_statista_with_internet['Internet_Users']
)

statista_per_user_start = df_statista_with_internet.iloc[0]['Events_per_User']
statista_per_user_end = df_statista_with_internet.iloc[-1]['Events_per_User']
statista_per_user_growth = ((statista_per_user_end - statista_per_user_start) / statista_per_user_start * 100)

print(f"\n" + "="*70)
print("6A - SUMMARY: Events per Internet User (Trend Analysis)")
print("="*70)
print(f"\n1. ADVISENS (2005-2023):")
print(f"   Events per user: {advisens_per_user_start:.2e} → {advisens_per_user_end:.2e}")
print(f"   Evolution: {advisens_per_user_growth:+.1f}%")
print(f"   → {'IMPROVING ✓' if advisens_per_user_growth < 0 else 'DETERIORATING ✗'} cyber environment per user")

print(f"\n2. STATISTA (2005-2023):")
print(f"   Events per user: {statista_per_user_start:.2e} → {statista_per_user_end:.2e}")
print(f"   Evolution: {statista_per_user_growth:+.1f}%")
print(f"   → {'IMPROVING ✓' if statista_per_user_growth < 0 else 'DETERIORATING ✗'} cyber environment per user")

print(f"\n3. MALWARE (2016-2024):")
print(f"   Attacks per user: {malware_per_user_start:.2e} → {malware_per_user_end:.2e}")
print(f"   Evolution: {malware_per_user_growth:+.1f}%")
print(f"   → {'IMPROVING ✓' if malware_per_user_growth < 0 else 'DETERIORATING ✗'} malware risk per user")

print(f"\n4. RANSOMWARE (2018-2023):")
print(f"   Attacks per user: {ransomware_per_user_start:.2e} → {ransomware_per_user_end:.2e}")
print(f"   Evolution: {ransomware_per_user_growth:+.1f}%")
print(f"   → {'IMPROVING ✓' if ransomware_per_user_growth < 0 else 'DETERIORATING ✗'} ransomware risk per user")

print(f"\n✅ Section 6A completed successfully!")


6A. SIMPLE NORMALIZATION: Events per Internet User
✓ World population data loaded from: c:\Users\jbertholat\Downloads\cyber-hazard-evolution\data\usage\dataworld_bank_world_population.csv
  Years available: 2004 - 2024

6A - SUMMARY: Events per Internet User (Trend Analysis)

1. ADVISENS (2005-2023):
   Events per user: 4.15e-13 → 2.14e-12
   Evolution: +416.3%
   → DETERIORATING ✗ cyber environment per user

2. STATISTA (2005-2023):
   Events per user: 1.34e-09 → 3.17e-09
   Evolution: +136.6%
   → DETERIORATING ✗ cyber environment per user

3. MALWARE (2016-2024):
   Attacks per user: 2.77e-06 → 1.15e-06
   Evolution: -58.5%
   → IMPROVING ✓ malware risk per user

4. RANSOMWARE (2018-2023):
   Attacks per user: 5.41e-08 → 6.02e-08
   Evolution: +11.2%
   → DETERIORATING ✗ ransomware risk per user

✅ Section 6A completed successfully!


In [159]:
# Visualisation 6a - Events per User (All four types)
fig_6a = go.Figure()

# Advisen per user
fig_6a.add_trace(go.Scatter(
    x=df_advisens_with_internet['Year'],
    y=df_advisens_with_internet['Events_per_User'],
    mode='lines+markers',
    name='Advisen (per user)',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8),
    hovertemplate='<b>%{x}</b><br>Advisen per user: %{y:.2e}<extra></extra>'
))

# Statista per user
fig_6a.add_trace(go.Scatter(
    x=df_statista_with_internet['Year'],
    y=df_statista_with_internet['Events_per_User'],
    mode='lines+markers',
    name='Statista (per user)',
    line=dict(color='#2ca02c', width=3),
    marker=dict(size=8),
    hovertemplate='<b>%{x}</b><br>Statista per user: %{y:.2e}<extra></extra>'
))

# Malware per user (scaled for visibility)
fig_6a.add_trace(go.Scatter(
    x=df_malware_with_internet['Year'],
    y=df_malware_with_internet['Malware_per_User'],
    mode='lines+markers',
    name='Malware (per user)',
    line=dict(color='#ff7f0e', width=3),
    marker=dict(size=8),
    hovertemplate='<b>%{x}</b><br>Malware per user: %{y:.2e}<extra></extra>'
))

# Ransomware per user
fig_6a.add_trace(go.Scatter(
    x=df_ransomware_with_internet['Year'],
    y=df_ransomware_with_internet['Ransomware_per_User'],
    mode='lines+markers',
    name='Ransomware (per user)',
    line=dict(color='#d62728', width=3),
    marker=dict(size=8),
    hovertemplate='<b>%{x}</b><br>Ransomware per user: %{y:.2e}<extra></extra>'
))

fig_6a.update_layout(
    title='<b>6a - Cyber Events per Internet User (Comparative View)</b>',
    xaxis_title='Year',
    yaxis_title='Events per Internet User (log scale)',
    yaxis_type='log',
    width=1200,
    height=650,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=150, b=100),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_6a.write_html(output_dir / '06a_events_per_user.html')
fig_6a.write_image(output_dir / '06a_events_per_user.png', width=1200, height=650)

print("\n✓ Chart saved: ./outputs/06a_events_per_user.html")
print("✓ Chart saved: ./outputs/06a_events_per_user.png")

fig_6a.show()


✓ Chart saved: ./outputs/06a_events_per_user.html
✓ Chart saved: ./outputs/06a_events_per_user.png


### 6b - Multivariate Regression (Elasticity Analysis)

**Methodology**: Log-log regression to measure event elasticity with respect to internet users.

**Formula**:
$$\log(I_t) = \alpha + \beta \log(U_t) + \varepsilon_t$$

Where:
- $I_t$ = Events at time t
- $U_t$ = Internet users at time t
- $\beta$ = Elasticity coefficient

**Interpretation**:
- $\beta = 1$: Events grow proportionally with users (pure exposure effect)
- $\beta > 1$: Events grow faster than users (structural risk increase)
- $\beta < 1$: Events grow slower than users (improved resilience/security)

In [160]:
print("\n" + "="*70)
print("6B. MULTIVARIATE REGRESSION: Elasticity Analysis (Log-Log)")
print("="*70)

# Function to perform log-log regression and return results
def log_log_regression(df, events_col, users_col, name):
    """Perform log-log regression on events vs internet users"""
    # Prepare data (remove any NaN or zero values)
    df_clean = df[[events_col, users_col]].dropna()
    df_clean = df_clean[df_clean[events_col] > 0]
    df_clean = df_clean[df_clean[users_col] > 0]
    
    # Log transformation
    X = np.log(df_clean[users_col].values)
    y = np.log(df_clean[events_col].values)
    
    # Linear regression in log space
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)
    
    # Calculate fitted values
    y_pred = slope * X + intercept
    
    results = {
        'name': name,
        'elasticity': slope,
        'intercept': intercept,
        'r_squared': r_value**2,
        'p_value': p_value,
        'std_err': std_err,
        'X': X,
        'y': y,
        'y_pred': y_pred,
        'df_clean': df_clean
    }
    
    return results

# Perform regression for each event type
advisens_reg = log_log_regression(df_advisens_with_internet, 'Events', 'Internet_Users', 'Advisen')
statista_reg = log_log_regression(df_statista_with_internet, 'Events', 'Internet_Users', 'Statista')
malware_reg = log_log_regression(df_malware_with_internet, 'Malware_Attacks', 'Internet_Users', 'Malware')
ransomware_reg = log_log_regression(df_ransomware_with_internet, 'Ransomware_Attacks', 'Internet_Users', 'Ransomware')

print(f"\n📊 ADVISENS - Log-Log Regression:")
print(f"   Elasticity (β): {advisens_reg['elasticity']:.4f}")
print(f"   Interpretation: {'Events grow slower than users (IMPROVING)' if advisens_reg['elasticity'] < 1 else 'Events grow faster than users (DETERIORATING)' if advisens_reg['elasticity'] > 1 else 'Events proportional to users (NEUTRAL)'}")
print(f"   R² (fit quality): {advisens_reg['r_squared']:.4f}")
print(f"   P-value: {advisens_reg['p_value']:.2e}")
print(f"   Standard Error: {advisens_reg['std_err']:.4f}")

print(f"\n📊 STATISTA - Log-Log Regression:")
print(f"   Elasticity (β): {statista_reg['elasticity']:.4f}")
print(f"   Interpretation: {'Events grow slower than users (IMPROVING)' if statista_reg['elasticity'] < 1 else 'Events grow faster than users (DETERIORATING)' if statista_reg['elasticity'] > 1 else 'Events proportional to users (NEUTRAL)'}")
print(f"   R² (fit quality): {statista_reg['r_squared']:.4f}")
print(f"   P-value: {statista_reg['p_value']:.2e}")
print(f"   Standard Error: {statista_reg['std_err']:.4f}")

print(f"\n📊 MALWARE - Log-Log Regression:")
print(f"   Elasticity (β): {malware_reg['elasticity']:.4f}")
print(f"   Interpretation: {'Malware grows slower than users (IMPROVING)' if malware_reg['elasticity'] < 1 else 'Malware grows faster than users (DETERIORATING)' if malware_reg['elasticity'] > 1 else 'Malware proportional to users (NEUTRAL)'}")
print(f"   R² (fit quality): {malware_reg['r_squared']:.4f}")
print(f"   P-value: {malware_reg['p_value']:.2e}")
print(f"   Standard Error: {malware_reg['std_err']:.4f}")

print(f"\n📊 RANSOMWARE - Log-Log Regression:")
print(f"   Elasticity (β): {ransomware_reg['elasticity']:.4f}")
print(f"   Interpretation: {'Ransomware grows slower than users (IMPROVING)' if ransomware_reg['elasticity'] < 1 else 'Ransomware grows faster than users (DETERIORATING)' if ransomware_reg['elasticity'] > 1 else 'Ransomware proportional to users (NEUTRAL)'}")
print(f"   R² (fit quality): {ransomware_reg['r_squared']:.4f}")
print(f"   P-value: {ransomware_reg['p_value']:.2e}")
print(f"   Standard Error: {ransomware_reg['std_err']:.4f}")

print(f"\n" + "="*70)
print("ELASTICITY COMPARISON")
print("="*70)
print(f"\nAdvisen β={advisens_reg['elasticity']:.4f} | Statista β={statista_reg['elasticity']:.4f} | Malware β={malware_reg['elasticity']:.4f} | Ransomware β={ransomware_reg['elasticity']:.4f}")
print(f"\nKey Insight:")
print(f"  • β < 1: Cyber risk IMPROVING relative to digital growth")
print(f"  • β ≈ 1: Cyber risk NEUTRAL (grows with exposure)")
print(f"  • β > 1: Cyber risk DETERIORATING faster than exposure")
print(f"\n✅ Section 6B completed successfully!")



6B. MULTIVARIATE REGRESSION: Elasticity Analysis (Log-Log)

📊 ADVISENS - Log-Log Regression:
   Elasticity (β): 1.6899
   Interpretation: Events grow faster than users (DETERIORATING)
   R² (fit quality): 0.8531
   P-value: 1.70e-08
   Standard Error: 0.1701

📊 STATISTA - Log-Log Regression:
   Elasticity (β): 3.0280
   Interpretation: Events grow faster than users (DETERIORATING)
   R² (fit quality): 0.9304
   P-value: 1.08e-04
   Standard Error: 0.3381

📊 MALWARE - Log-Log Regression:
   Elasticity (β): -0.8691
   Interpretation: Malware grows slower than users (IMPROVING)
   R² (fit quality): 0.4925
   P-value: 3.51e-02
   Standard Error: 0.3334

📊 RANSOMWARE - Log-Log Regression:
   Elasticity (β): 2.3282
   Interpretation: Ransomware grows faster than users (DETERIORATING)
   R² (fit quality): 0.6284
   P-value: 3.35e-02
   Standard Error: 0.8007

ELASTICITY COMPARISON

Advisen β=1.6899 | Statista β=3.0280 | Malware β=-0.8691 | Ransomware β=2.3282

Key Insight:
  • β < 1: Cyber r

In [161]:
# Visualisation 6b - Log-Log Regression Plot for all three types
fig_6b = make_subplots(
    rows=1, cols=4,
    subplot_titles=('Advisen', 'Statista', 'Malware', 'Ransomware'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}]]
)

regs = [advisens_reg, statista_reg, malware_reg, ransomware_reg]
colors = ['#1f77b4', '#ff7f0e', '#d62728', '#2ca02c']
col_pos = [1, 2, 3, 4]
pos_map = {1: 'br', 2: 'br', 3: 'bl', 4: 'tl'}


for reg, color, col in zip(regs, colors, col_pos):
    # Data points
    fig_6b.add_trace(
        go.Scatter(
            x=np.exp(reg['X']),
            y=np.exp(reg['y']),
            mode='markers',
            name=f"{reg['name']} (data)",
            marker=dict(color=color, size=8, opacity=0.6),
            hovertemplate=f"<b>{reg['name']}</b><br>Users: %{{x:.2e}}<br>Events: %{{y:.2e}}<extra></extra>",
            showlegend=False
        ),
        row=1, col=col
    )
    
    # Regression line
    X_sorted = np.sort(reg['X'])
    y_pred_sorted = reg['intercept'] + reg['elasticity'] * X_sorted
    
    fig_6b.add_trace(
        go.Scatter(
            x=np.exp(X_sorted),
            y=np.exp(y_pred_sorted),
            mode='lines',
            name=f"{reg['name']} (β={reg['elasticity']:.3f})",
            line=dict(color=color, width=3, dash='dash'),
            hovertemplate=f"<b>Regression: β={reg['elasticity']:.3f}</b><br>Users: %{{x:.2e}}<br>Predicted: %{{y:.2e}}<extra></extra>",
            showlegend=False
        ),
        row=1, col=col
    )
    
    # Update axes to log scale
    fig_6b.update_xaxes(type='log', row=1, col=col)
    fig_6b.update_yaxes(type='log', row=1, col=col)

fig_6b.update_xaxes(title_text="Internet Users (log scale)")
fig_6b.update_yaxes(title_text="Events (log scale) (Millions)", row=1, col=1)

fig_6b.update_layout(
    title={
        'text': '<b>Log-Log Regression: Event Elasticity vs Internet Users</b>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': '#222222'}
    },
    height=600,
    width=1400,
    showlegend=False,
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=80, r=80, t=150, b=80)
)


# --- Annotations régression en coordonnées d'axes log (log10) ---
pos_map = {1: 'br', 2: 'br', 3: 'bl', 4: 'tl'}  # 1–2 bas-droite, 3 bas-gauche, 4 haut-gauche
annotations_to_add = []

# petite sécurité pour éviter log10(0) ou négatif
def safe_log10(v, default=-3.0):
    try:
        return math.log10(v) if v > 0 else default
    except Exception:
        return default

for reg, color, col in zip(regs, colors, col_pos):
    # données en espace naturel
    x_data = np.exp(reg['X'])
    y_data = np.exp(reg['y'])

    # si valeurs non-positives, on saute (axes log exigent > 0)
    if np.any(x_data <= 0) or np.any(y_data <= 0):
        continue

    xmin, xmax = float(np.min(x_data)), float(np.max(x_data))
    ymin, ymax = float(np.min(y_data)), float(np.max(y_data))

    # marges multiplicatives pour "décoller" du bord (robuste en log)
    fx = (xmax / xmin) ** 0.10 if xmax > xmin and xmin > 0 else 1.5
    fy = (ymax / ymin) ** 0.10 if ymax > ymin and ymin > 0 else 1.5

    pos = pos_map[col]
    if pos == 'br':      # bas-droite
        x_nat = xmax / fx
        y_nat = ymin * fy
        xanchor, yanchor, align = 'right', 'bottom', 'left'
    elif pos == 'bl':    # bas-gauche
        x_nat = xmin * fx
        y_nat = ymin * fy
        xanchor, yanchor, align = 'left', 'bottom', 'left'
    else:                # 'tl' haut-gauche
        x_nat = xmin * fx
        y_nat = ymax / fy
        xanchor, yanchor, align = 'left', 'top', 'left'

    # conversion en coordonnées log10 car xref/yref = axes (et axes sont en log)
    x_log = safe_log10(x_nat)
    y_log = safe_log10(y_nat)

    # références d’axes du bon sous-graphe
    xref = 'x' if col == 1 else f'x{col}'
    yref = 'y' if col == 1 else f'y{col}'

    # texte de la boîte
    ann_text = (
        f"β = {reg['elasticity']:.3f}<br>"
        f"α = {reg['intercept']:.3f}<br>"
        f"R² = {reg['r_squared']:.4f}<br>"
        f"sample length = {len(reg['X'])}"
    )

    annotations_to_add.append(dict(
        x=x_log, y=y_log, xref=xref, yref=yref,   # <-- positions en log10
        text=ann_text, showarrow=False,
        xanchor=xanchor, yanchor=yanchor, align=align,
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor=color, borderwidth=1,
        font=dict(size=12, color='#222')
    ))

# préserver d’éventuelles annotations existantes (ex. titres de sous-graphe)
existing_anns = list(fig_6b.layout.annotations) if fig_6b.layout.annotations else []
fig_6b.update_layout(annotations=existing_anns + annotations_to_add)

fig_6b.write_html(output_dir / '06b_log_log_regression.html')
fig_6b.write_image(output_dir / '06b_log_log_regression.png', width=1400, height=600)

print("\n✓ Chart saved: ./outputs/06b_log_log_regression.html")
print("✓ Chart saved: ./outputs/06b_log_log_regression.png")

fig_6b.show()


✓ Chart saved: ./outputs/06b_log_log_regression.html
✓ Chart saved: ./outputs/06b_log_log_regression.png


### 6c - Residual Analysis: Structural Risk Evolution at Constant Exposure

**Methodology**: Extract residuals from the log-log regression to identify structural risk evolutions independent of internet exposure.

**Formula**:
$$\varepsilon_t = \log(I_t) - [\alpha + \beta \log(U_t)]$$

**Interpretation**:
- Residuals > 0: More events than expected for that internet exposure level (structural deterioration)
- Residuals ≈ 0: Events match expected level (stable structural risk)
- Residuals < 0: Fewer events than expected (structural improvement/increased resilience)
- Trend in residuals: Evolution of structural cyber risk independent of digital exposure

In [162]:
print("\n" + "="*70)
print("6C. RESIDUAL ANALYSIS: Structural Risk Evolution")
print("="*70)

# Function to extract residuals and analyze trends
def analyze_residuals(df, events_col, users_col, years_col, reg_results, name):
    """Extract and analyze residuals from regression"""
    df_clean = df[[events_col, users_col, years_col]].dropna()
    df_clean = df_clean[df_clean[events_col] > 0]
    df_clean = df_clean[df_clean[users_col] > 0]
    
    # Calculate residuals
    X_log = np.log(df_clean[users_col].values)
    y_actual_log = np.log(df_clean[events_col].values)
    y_predicted_log = reg_results['intercept'] + reg_results['elasticity'] * X_log
    residuals = y_actual_log - y_predicted_log
    
    # Residuals in original space (exponential)
    residuals_percent = (np.exp(residuals) - 1) * 100
    
    # Trend analysis of residuals
    years_array = df_clean[years_col].values.astype(float)
    res_slope, res_intercept, res_r, res_p, res_err = stats.linregress(years_array, residuals)
    
    results_dict = {
        'years': years_array,
        'residuals': residuals,
        'residuals_percent': residuals_percent,
        'slope': res_slope,
        'r_squared': res_r**2,
        'p_value': res_p,
        'interpretation': 'improving' if res_slope < 0 else 'deteriorating' if res_slope > 0 else 'stable'
    }
    
    return results_dict

# Analyze residuals for each event type
advisens_residuals = analyze_residuals(df_advisens_with_internet, 'Events', 'Internet_Users', 'Year', advisens_reg, 'Advisen')
statista_resuiduals = analyze_residuals(df_statista_with_internet, 'Events', 'Internet_Users', 'Year', statista_reg, 'Statista')
malware_residuals = analyze_residuals(df_malware_with_internet, 'Malware_Attacks', 'Internet_Users', 'Year', malware_reg, 'Malware')
ransomware_residuals = analyze_residuals(df_ransomware_with_internet, 'Ransomware_Attacks', 'Internet_Users', 'Year', ransomware_reg, 'Ransomware')

print(f"\n📊 ADVISENS - Residual Analysis:")
print(f"   Mean Residual: {np.mean(advisens_residuals['residuals']):.4f} (log scale)")
print(f"   Mean Residual: {np.mean(advisens_residuals['residuals_percent']):+.1f}% (percentage)")
print(f"   Trend Slope: {advisens_residuals['slope']:.6f}/year")
print(f"   Trend: {advisens_residuals['interpretation'].upper()}")
print(f"   P-value: {advisens_residuals['p_value']:.4f}")
print(f"   Interpretation: Structural cyber risk is {advisens_residuals['interpretation']}")

print(f"\n📊 STATISTA - Residual Analysis:")
print(f"   Mean Residual: {np.mean(statista_resuiduals['residuals']):.4f} (log scale)")
print(f"   Mean Residual: {np.mean(statista_resuiduals['residuals_percent']):+.1f}% (percentage)")
print(f"   Trend Slope: {statista_resuiduals['slope']:.6f}/year")
print(f"   Trend: {statista_resuiduals['interpretation'].upper()}")
print(f"   P-value: {statista_resuiduals['p_value']:.4f}")
print(f"   Interpretation: Structural cyber risk is {statista_resuiduals['interpretation']}")

print(f"\n📊 MALWARE - Residual Analysis:")
print(f"   Mean Residual: {np.mean(malware_residuals['residuals']):.4f} (log scale)")
print(f"   Mean Residual: {np.mean(malware_residuals['residuals_percent']):+.1f}% (percentage)")
print(f"   Trend Slope: {malware_residuals['slope']:.6f}/year")
print(f"   Trend: {malware_residuals['interpretation'].upper()}")
print(f"   P-value: {malware_residuals['p_value']:.4f}")
print(f"   Interpretation: Structural malware risk is {malware_residuals['interpretation']}")

print(f"\n📊 RANSOMWARE - Residual Analysis:")
print(f"   Mean Residual: {np.mean(ransomware_residuals['residuals']):.4f} (log scale)")
print(f"   Mean Residual: {np.mean(ransomware_residuals['residuals_percent']):+.1f}% (percentage)")
print(f"   Trend Slope: {ransomware_residuals['slope']:.6f}/year")
print(f"   Trend: {ransomware_residuals['interpretation'].upper()}")
print(f"   P-value: {ransomware_residuals['p_value']:.4f}")
print(f"   Interpretation: Structural ransomware risk is {ransomware_residuals['interpretation']}")

print(f"\n" + "="*70)
print("STRUCTURAL RISK INTERPRETATION")
print("="*70)
print(f"""
The residuals show how much events deviate from what internet exposure alone
would predict. A positive residual trend indicates that cyber events are
increasing faster than the model would expect from internet growth alone,
suggesting structural deterioration in the cyber threat landscape.
""")

print(f"\n✅ Section 6C completed successfully!")



6C. RESIDUAL ANALYSIS: Structural Risk Evolution

📊 ADVISENS - Residual Analysis:
   Mean Residual: -0.0000 (log scale)
   Mean Residual: +5.9% (percentage)
   Trend Slope: -0.006095/year
   Trend: IMPROVING
   P-value: 0.6949
   Interpretation: Structural cyber risk is improving

📊 STATISTA - Residual Analysis:
   Mean Residual: 0.0000 (log scale)
   Mean Residual: +1.1% (percentage)
   Trend Slope: -0.009109/year
   Trend: IMPROVING
   P-value: 0.7331
   Interpretation: Structural cyber risk is improving

📊 MALWARE - Residual Analysis:
   Mean Residual: -0.0000 (log scale)
   Mean Residual: +1.6% (percentage)
   Trend Slope: 0.002231/year
   Trend: DETERIORATING
   P-value: 0.9333
   Interpretation: Structural malware risk is deteriorating

📊 RANSOMWARE - Residual Analysis:
   Mean Residual: -0.0000 (log scale)
   Mean Residual: +3.8% (percentage)
   Trend Slope: -0.010124/year
   Trend: IMPROVING
   P-value: 0.8730
   Interpretation: Structural ransomware risk is improving

STRUCTU

In [163]:
# Visualisation 6c - Residuals Over Time
fig_6c = go.Figure()
plot_all = False  # Set to False to plot only Advisen and Statista

# Advisen residuals
fig_6c.add_trace(go.Scatter(
    x=advisens_residuals['years'],
    y=advisens_residuals['residuals_percent'],
    mode='lines+markers',
    name='Advisen',
    line=dict(color='#1f77b4', width=2.5),
    marker=dict(size=7),
    fill='tozeroy',
    opacity=0.3,
    hovertemplate='<b>Advisen</b><br>Year: %{x}<br>Residual: %{y:+.1f}%<extra></extra>'
))

# Add trend line for Advisen
advisens_trend = advisens_residuals['slope'] * advisens_residuals['years'] + (
    np.mean(advisens_residuals['residuals_percent']) - advisens_residuals['slope'] * np.mean(advisens_residuals['years'])
)
fig_6c.add_trace(go.Scatter(
    x=advisens_residuals['years'],
    y=advisens_trend,
    mode='lines',
    name='Advisen Trend',
    line=dict(color='#1f77b4', width=3, dash='dash'),
    hovertemplate='<b>Advisen Trend</b><br>Year: %{x}<br>Trend: %{y:+.1f}%<extra></extra>'
))

#Statista residuals
fig_6c.add_trace(go.Scatter(
    x=statista_resuiduals['years'],
    y=statista_resuiduals['residuals_percent'],
    mode='lines+markers',
    name='Statista',
    line=dict(color='#2ca02c', width=2.5),
    marker=dict(size=7),
    fill='tozeroy',
    opacity=0.3,
    hovertemplate='<b>Statista</b><br>Year: %{x}<br>Residual: %{y:+.1f}%<extra></extra>'
))

#Add trend line for Statista
statista_trend = statista_resuiduals['slope'] * statista_resuiduals['years'] + (
    np.mean(statista_resuiduals['residuals_percent']) - statista_resuiduals['slope'] * np.mean(statista_resuiduals['years'])
)
fig_6c.add_trace(go.Scatter(
    x=statista_resuiduals['years'],
    y=statista_trend,
    mode='lines',
    name='Statista Trend',
    line=dict(color='#2ca02c', width=3, dash='dash'),
    hovertemplate='<b>Statista Trend</b><br>Year: %{x}<br>Trend: %{y:+.1f}%<extra></extra>'
))

if plot_all:
    # Malware residuals
    fig_6c.add_trace(go.Scatter(
        x=malware_residuals['years'],
        y=malware_residuals['residuals_percent'],
        mode='lines+markers',
        name='Malware',
        line=dict(color='#ff7f0e', width=2.5),
        marker=dict(size=7),
        fill='tozeroy',
        opacity=0.3,
        hovertemplate='<b>Malware</b><br>Year: %{x}<br>Residual: %{y:+.1f}%<extra></extra>'
    ))

    # Add trend line for Malware
    malware_trend = malware_residuals['slope'] * malware_residuals['years'] + (
        np.mean(malware_residuals['residuals_percent']) - malware_residuals['slope'] * np.mean(malware_residuals['years'])
    )
    fig_6c.add_trace(go.Scatter(
        x=malware_residuals['years'],
        y=malware_trend,
        mode='lines',
        name='Malware Trend',
        line=dict(color='#ff7f0e', width=3, dash='dash'),
        hovertemplate='<b>Malware Trend</b><br>Year: %{x}<br>Trend: %{y:+.1f}%<extra></extra>'
    ))

    # Ransomware residuals
    fig_6c.add_trace(go.Scatter(
        x=ransomware_residuals['years'],
        y=ransomware_residuals['residuals_percent'],
        mode='lines+markers',
        name='Ransomware',
        line=dict(color='#d62728', width=2.5),
        marker=dict(size=7),
        fill='tozeroy',
        opacity=0.3,
        hovertemplate='<b>Ransomware</b><br>Year: %{x}<br>Residual: %{y:+.1f}%<extra></extra>'
    ))

    # Add trend line for Ransomware
    ransomware_trend = ransomware_residuals['slope'] * ransomware_residuals['years'] + (
        np.mean(ransomware_residuals['residuals_percent']) - ransomware_residuals['slope'] * np.mean(ransomware_residuals['years'])
    )
    fig_6c.add_trace(go.Scatter(
        x=ransomware_residuals['years'],
        y=ransomware_trend,
        mode='lines',
        name='Ransomware Trend',
        line=dict(color='#d62728', width=3, dash='dash'),
        hovertemplate='<b>Ransomware Trend</b><br>Year: %{x}<br>Trend: %{y:+.1f}%<extra></extra>'
    ))

# Add zero line as reference
fig_6c.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Expected level (zero residual)")

fig_6c.update_layout(
    title={
        'text': '<b>Residual Analysis: Structural Risk Evolution Over Time</b><br><sub>Deviation from expected events based on internet exposure alone</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': '#222222'}
    },

    xaxis_title='Year',
    yaxis_title='Residual (% deviation from expected)',
    width=1200,
    height=700,
    hovermode='x unified',
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff',
    margin=dict(l=100, r=100, t=180, b=100),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
    annotations=[
        dict(
            text='Positive residuals = More events than expected for exposure level<br>Negative residuals = Fewer events than expected (improvement)',
            xref='paper', yref='paper',
            x=0.98, y=-0.12,
            xanchor='right',
            showarrow=False,
            font=dict(size=11, color='#555555')
        )
    ]
)

fig_6c.write_html(output_dir / '06c_residual_analysis.html')
fig_6c.write_image(output_dir / '06c_residual_analysis.png', width=1200, height=700)

print("\n✓ Chart saved: ./outputs/06c_residual_analysis.html")
print("✓ Chart saved: ./outputs/06c_residual_analysis.png")

fig_6c.show()


✓ Chart saved: ./outputs/06c_residual_analysis.html
✓ Chart saved: ./outputs/06c_residual_analysis.png


### Summary: Integration of Three Comparative Approaches

**6a - Simple Normalization**: Quick view of risk per internet user
- Shows whether the ratio of events to users is improving or deteriorating
- Easy to understand but doesn't control for non-linearity

**6b - Elasticity Regression**: Identifies relationship structure
- Measures how sensitive events are to internet user growth
- Reveals if growth is proportional (β≈1), accelerating (β>1), or decelerating (β<1)
- Provides statistical confidence in the relationship

**6c - Residual Analysis**: Isolates structural risk evolution
- Shows anomalies from expected pattern
- Separates structural evolutions from exposure effects
- Trend in residuals reveals if cyber environment is intrinsically improving or deteriorating